In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torch.nn as nn
from torchvision import transforms

In [2]:
transformer=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])
label_to_num={"ants":0,"bees":1}

In [3]:
# tep_path='data/hymenoptera_data/train/ants/0013035.jpg'
# img=Image.open(tep_path)
# img=transformer(img)
# print(img.shape)

In [4]:
class MyData(Dataset):

    def __init__(self, root_dir,label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path=os.path.join(self.root_dir,self.label_dir)
        self.image_path=os.listdir(self.path)

    def __getitem__(self, idx):
        image_name=self.image_path[idx]
        image=Image.open(os.path.join(self.path,image_name)).convert('RGB')
        label=self.label_dir
        return transformer(image), label_to_num[label]

    def __len__(self):
        return len(self.image_path)

In [5]:
root_dir='data/hymenoptera_data/train'
ants_label_dir='ants'
bees_label_dir='bees'
ants_dataset=MyData(root_dir,ants_label_dir)
bees_dataset=MyData(root_dir,bees_label_dir)
all_dataset=ants_dataset + bees_dataset
dataloader=DataLoader(all_dataset,batch_size=64,shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
root_val='data/hymenoptera_data/val'
ants_val=MyData(root_val,ants_label_dir)
bees_val=MyData(root_val,bees_label_dir)
all_val=ants_val + bees_val
val_loader=DataLoader(all_val,batch_size=64,shuffle=True)

In [7]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Conv2d(3,32,7,1,3),
            nn.MaxPool2d(4),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(4),
            nn.Conv2d(64,128,3,1,1),
            nn.MaxPool2d(2),
            nn.Conv2d(128,128,3,1,1),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(2048,256),
            nn.ReLU(),
            nn.Linear(256,16),
            nn.ReLU(),
            nn.Linear(16,2)
        )

    def forward(self, x):
        return self.net(x)
my_model=MyModule()

In [8]:
my_model.to(device)
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)
optimizer = torch.optim.Adam(my_model.parameters(),lr=0.001,weight_decay=0.01)

In [9]:
from torch.utils.tensorboard import SummaryWriter

epochs = 40
train_step=0
total_test_step=0
test_data_size = len(all_val)
writer = SummaryWriter("my_dl_logs")

In [10]:
for i in range(epochs):
    print('epoch',i+1)
    my_model.train()

    for data in dataloader:

        image,label=data
        image=image.to(device)
        label=label.to(device)
        outputs=my_model(image)

        loss=loss_fn(outputs,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_step+=1
        writer.add_scalar('train loss',loss.item(),train_step)
        if(train_step%10==0):
            print(f"step:{train_step} train loss:{loss.item():.4f}")

    my_model.eval()
    total_test_loss=0
    total_accuracy=0
    with torch.no_grad():
        for data in val_loader:
            image,label=data
            image=image.to(device)
            label=label.to(device)
            outputs=my_model(image)
            loss=loss_fn(outputs,label)
            total_test_loss+=loss.item()
            accuracy = (outputs.argmax(1) == label).sum()
            total_accuracy+=accuracy

    print("整体测试集上的Loss：{}".format(total_test_loss/len(val_loader)))
    print("整体测试集上的正确率：{}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss",total_test_loss/len(val_loader),total_test_step)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,total_test_step)
    total_test_step = total_test_step + 1


epoch 1
整体测试集上的Loss：0.6724225282669067
整体测试集上的正确率：0.601307213306427
epoch 2
整体测试集上的Loss：0.6854997078577677
整体测试集上的正确率：0.5751634240150452
epoch 3
step:10 train loss:0.6467
整体测试集上的Loss：0.6548200845718384
整体测试集上的正确率：0.601307213306427
epoch 4
整体测试集上的Loss：0.6600675185521444
整体测试集上的正确率：0.5686274766921997
epoch 5
step:20 train loss:0.6250
整体测试集上的Loss：0.7686635653177897
整体测试集上的正确率：0.5882353186607361
epoch 6
整体测试集上的Loss：0.6774961352348328
整体测试集上的正确率：0.5359477400779724
epoch 7
整体测试集上的Loss：0.7637085715929667
整体测试集上的正确率：0.5816993713378906
epoch 8
step:30 train loss:0.5422
整体测试集上的Loss：0.6843770742416382
整体测试集上的正确率：0.5751634240150452
epoch 9
整体测试集上的Loss：0.6361744403839111
整体测试集上的正确率：0.6143791079521179
epoch 10
step:40 train loss:0.4180
整体测试集上的Loss：0.8410924474398295
整体测试集上的正确率：0.5947712659835815
epoch 11
整体测试集上的Loss：0.8193130691846212
整体测试集上的正确率：0.6143791079521179
epoch 12
整体测试集上的Loss：0.8031263550122579
整体测试集上的正确率：0.516339898109436
epoch 13
step:50 train loss:0.5840
整体测试集上的Loss：0.7686176697413126
整体

In [11]:
writer.close()

In [12]:
# for data in dataloader:
#     image,label=data
#     image=image.to(device)
#     label=label.to(device)
#     print(my_model(image).shape)
#     break